In [1]:
#from : https://www.kaggle.com/code/suyue715/tfidfvectorizer/notebook dengan beberapa modifikasi karena method berubah nama di versi baru 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from nltk.corpus import stopwords #buat pake tools stopwords punya nltk
from nltk.tokenize import word_tokenize #buat pake tools tokenization punya nltk
from nltk.stem import SnowballStemmer #buat pake tools stemmer (ngubah kata jadi kata dasar)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output #ini buat cek yang ada di direktori apa aja
print(check_output(["ls", "../input"]).decode("utf8"))

#sama-sama tools NLP, yang pertama spacy yang kedua teks blob, nanti bisa bantu saat proses tokenisasi stemming dkk, 
#nanti pakai yang library EN karena bahasa penulis horornya pakai EN
import spacy #buat import spacy 
from textblob import TextBlob

# Any results you write to the current directory are saved as output.
from keras.preprocessing import sequence #import sequence buat model RNN/LSTM
from tensorflow.keras.utils import to_categorical #impor fungsi mengubah label ke format one-hot encoding
from keras.models import Sequential #buat model jaringan secara urut (satu-satu gitu)
from keras.layers import Dense, Dropout, Activation, Embedding #buat jaringan saraf
from keras.layers import LSTM #impor layer LSTM buat proses data secara sekuensial

from sklearn.ensemble import RandomForestClassifier #buat klasifikasi berbasis banyak decision tree
from sklearn.model_selection import train_test_split #buat pisahin data menjadi data train dan test
from sklearn.metrics import confusion_matrix, classification_report #hitung confusion matriks sama buat report klasifikasi
from sklearn.feature_extraction.text import CountVectorizer #ubah teks ke vektor berdasar jumlah kata
from sklearn.decomposition import LatentDirichletAllocation #LDA untuk topik modelling
from sklearn.feature_extraction.text import TfidfVectorizer #mengubah teks menjadi vektor berdasar TF-IDF
from sklearn.decomposition import TruncatedSVD #buat reduksi dimensi setelah pake TF-IDF

ir-2025-spooky-author-identification



2025-05-14 11:34:11.983367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747222452.288981      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747222452.379929      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
#Buat load datasets
train = pd.read_csv('/kaggle/input/ir-2025-spooky-author-identification/train/train.csv')
test = pd.read_csv('/kaggle/input/ir-2025-spooky-author-identification/test/test.csv')
sample = pd.read_csv('/kaggle/input/ir-2025-spooky-author-identification/sample_submission/sample_submission.csv')

# Preprocessing

In [4]:
#Ini buat assign author ke kolom baru 'author_num'
#0 -> 'EAP'
#1 -> 'HPL'
#2 -> 'MWS'
train['author_num']=train['author'].apply({'EAP':0,  'HPL':1,'MWS':2}.get)
train.head() #ini buat munculin datanya

,id,text,author,author_num
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


In [5]:
#Ini buat assign feature sama targetnya
X_text_train=train['text'].values
X_text_test=test['text'].values
y=train['author_num'].values
num_labels = len(np.unique(train['author_num']))

In [6]:
#Ini buat define stopword dan buat stemming , librarynya pakai bahasa EN
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
stemmer = SnowballStemmer('english')

# NLTK tokenize package

In [7]:
#Preprocess teks di train 
processed_train = []
for doc in X_text_train:
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in filtered]
    processed_train.append(stemmed)
#Preprocess teks di test
processed_test = []
for doc in X_text_test:
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in filtered]
    processed_test.append(stemmed)

In [8]:
#ini buat coba X_text_train / testing output. Output yang muncul itu indeks 1 dari X_text_train
X_text_train[1]

'It never once occurred to me that the fumbling might be a mere mistake.'

In [9]:
#ini buat coba tes processed_train / testing output. Output itu indeks 1 dari X_text_train yang udah di tokenisasi, udah di filter dari stopwords dan udah di stem (diubah ke bentuk dasar)
processed_train[1]

['it', 'never', 'occur', 'fumbl', 'might', 'mere', 'mistak']

In [10]:
#ini buat nambah kolom processed_train ke tabel test 
train['processed_train']=processed_train

In [11]:
#ini buat tes apakah kolom baru processed_train sudah masuk
train.head()

,id,text,author,author_num,processed_train
0,id26305,"This process, however, afforded me no means of...",EAP,0,"[this, process, howev, afford, mean, ascertain..."
1,id17569,It never once occurred to me that the fumbling...,HPL,1,"[it, never, occur, fumbl, might, mere, mistak]"
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0,"[in, left, hand, gold, snuff, box, caper, hill..."
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2,"[how, love, spring, as, look, windsor, terrac,..."
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1,"[find, noth, els, even, gold, superintend, aba..."


In [12]:
#ini buat tau kolom apa aja yang udah ada di tabel train
train.columns

Index(['id', 'text', 'author', 'author_num', 'processed_train'], dtype='object')

In [13]:
#menggabungkan token hasil pre-processing menjadi kalimat utuh kemudian dijadikan satu di 'final_processed_text' di tabel train

row_lst = []
for lst in train.loc[:,'processed_train']:
    text = ''
    for word in lst:
        text = text + ' ' + word
    row_lst.append(text)

train['final_processed_text'] = row_lst

In [14]:
#memasukkan kolom 'processed_test' di tabel test
test['processed_test']=processed_test
test.head()

,id,text,processed_test
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa..."
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, could, readili, fan, new..."
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ..."
3,id27757,While I was thinking how I should possibly man...,"[while, i, think, i, possibl, manag, without, ..."
4,id04081,I am not sure to what limit his knowledge may ...,"[i, sure, limit, knowledg, may, extend]"


In [15]:
#menggabungkan token hasil pre-processing menjadi kalimat utuh kemudian dijadikan satu di 'final_processed_text' di tabel test
#memasukkan kolom 'final_processed_test' ke tabel test
row_lst = []
for lst in test.loc[:,'processed_test']:
    text = ''
    for word in lst:
        text = text + ' ' + word
    row_lst.append(text)

test['final_processed_test'] = row_lst

test.head()

,id,text,processed_test,final_processed_test
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fat...
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap go...
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hu...
3,id27757,While I was thinking how I should possibly man...,"[while, i, think, i, possibl, manag, without, ...",while i think i possibl manag without one act...
4,id04081,I am not sure to what limit his knowledge may ...,"[i, sure, limit, knowledg, may, extend]",i sure limit knowledg may extend


In [16]:
#buat nge tes tabel train kolomnya udah ketambah semua atau belum
train.head()

,id,text,author,author_num,processed_train,final_processed_text
0,id26305,"This process, however, afforded me no means of...",EAP,0,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dime...
1,id17569,It never once occurred to me that the fumbling...,HPL,1,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut ma...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixtee...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon a...


# Named Entities

In [17]:
#ini buat install en_core_web_sm buat spacy
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 974, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
             

In [18]:
#ini buat nge load en_core_web_sm
nlp = spacy.load('en_core_web_sm')

content = []
for i in train['processed_train']:
    content.append(i)


# Use LDA to identify topic

In [19]:
# menjalankan Topic Modeling dengan metode LDA (Latent Dirichlet Allocation) pada data teks menggunakan CountVectorizer dari Scikit-lear
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD #ini buat import
cv = CountVectorizer(stop_words='english') #ini buat ngatur stopwordnya dari EN
cv.fit(train['text']) #ini buat train cv dari kolom teks
X = cv.transform(train['text']) #ini buat ngubah dari teks ke matriks
feature_names = cv.get_feature_names_out() #ini mengambil daftar kata yang dipake cv

lda = LatentDirichletAllocation(n_components=10) #ini buat lda dengan topiknya ada 10
lda.fit(X) #melatih LDA

#ini buat simpan result ke dataframe
results = pd.DataFrame(lda.components_,
                      columns=feature_names)

#ini intinya nge print
for topic in range(10):
    print('Topic', topic)
    word_list = results.T[topic].sort_values(ascending=False).index
    print(' '.join(word_list[0:25]), '\n')

Topic 0
time man long door saw eyes great came light open house feet room say head did box thing floor place thought left length near wall 

Topic 1
old man door adrian away time day eyes left night saw said room life hand long human idris city days like house raymond come turned 

Topic 2
time little think old said mind like thing did man saw took house day idea reason new home went believe matter world knew place night 

Topic 3
love said voice eyes death father heart words mother hand years know man heard life did shall sweet world thy like came length child hope 

Topic 4
mr did old thing said quite soon time things heart really long room life great friend fear know having tell man death understand eyes case 

Topic 5
said little great let place point like say way thought man face shall time life small general madame appeared mind marie degree make answer respect 

Topic 6
shall did day life say things like let death come great saw long time heard human strange came friend house mo

# RandomForestClassifier

In [20]:
#membagi dataset menjadi data pelatihan (train) dan data pengujian (test)
X_train, X_test, y_train, y_test = train_test_split(train['final_processed_text'],
                                                   train['author_num'],
                                                   test_size=0.33,
                                                   random_state=8675309)

In [21]:
#ini dipake buat bangun dan eval model klasifikasi pake random forest
cv = CountVectorizer(stop_words='english')
cv.fit(X_train)

X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

rf = RandomForestClassifier()
rf.fit(X_train_cv, y_train)
print(rf.score(X_test_cv, y_test))
predictions = rf.predict(X_test_cv)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.7079851439182916
[[1950  278  434]
 [ 423 1156  247]
 [ 363  142 1469]]
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      2662
           1       0.73      0.63      0.68      1826
           2       0.68      0.74      0.71      1974

    accuracy                           0.71      6462
   macro avg       0.71      0.70      0.70      6462
weighted avg       0.71      0.71      0.71      6462



# TfidfVectorizer() with English stop words

In [22]:
#ini intinya buat hitung tfidf
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(X_train)

X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
test_tfidf = tfidf.transform(test['final_processed_test'])

rf = RandomForestClassifier()
rf.fit(X_train_tfidf, y_train)
print(rf.score(X_test_tfidf, y_test))
predictions = rf.predict(X_test_tfidf)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.7140204271123491
[[2034  277  351]
 [ 417 1226  183]
 [ 427  193 1354]]
              precision    recall  f1-score   support

           0       0.71      0.76      0.73      2662
           1       0.72      0.67      0.70      1826
           2       0.72      0.69      0.70      1974

    accuracy                           0.71      6462
   macro avg       0.72      0.71      0.71      6462
weighted avg       0.71      0.71      0.71      6462



In [23]:
pred=rf.predict_proba(test_tfidf) #predict probabilitas
prob=pd.DataFrame(pred,columns=['EAP','HPL','MWS']) #buat kolom
submit1=pd.concat([test, prob], axis=1) #buat gabungin test sama prob
del submit1['text'] #hapus kolom text dari submit1

In [24]:
#hapus ['processed_test'] dari submit1
del submit1['processed_test']

In [25]:
#hapus ['final_processed_test'] dari submit1
del submit1['final_processed_test']

In [26]:
#hasil akhir
submit1

,id,EAP,HPL,MWS
0,id02310,0.140000,0.060000,0.800000
1,id24541,0.743333,0.151667,0.105000
2,id00134,0.290000,0.660000,0.050000
3,id27757,0.456250,0.340625,0.203125
4,id04081,0.827972,0.069528,0.102500
...,...,...,...,...
8387,id11749,0.712882,0.033628,0.253489
8388,id10526,0.409947,0.123939,0.466114
8389,id13477,0.700000,0.150000,0.150000
8390,id13761,0.230000,0.180000,0.590000


In [27]:
#buat submission
submit1.to_csv('./kelompok2ver1.csv', index=False, header=True)